# RAG 평가(Evaluation) 튜토리얼 - Part 2: LLM-as-a-Judge

## 왜 평가가 RAG 아키텍처보다 더 중요한가?

RAG(Retrieval-Augmented Generation) 시스템을 구축할 때 많은 개발자들이 **아키텍처 선택**에 집중합니다:
- 어떤 임베딩 모델을 사용할까?
- 어떤 벡터 데이터베이스가 좋을까?
- 청킹 전략은 어떻게 할까?

하지만 진짜 중요한 질문은 따로 있습니다:

> **"우리 RAG 시스템이 실제로 잘 작동하는지 어떻게 알 수 있는가?"**

### 평가가 중요한 이유

| 관점 | 평가 없이 | 평가 있으면 |
|------|----------|------------|
| **개발** | 감으로 튜닝 | 데이터 기반 개선 |
| **배포** | 불안한 출시 | 자신있는 릴리즈 |
| **비즈니스** | ROI 측정 불가 | 명확한 성과 지표 |
| **유지보수** | 회귀 버그 발견 어려움 | 지속적인 품질 모니터링 |

### 비즈니스 관점에서의 평가 필요성

1. **비용 정당화**: RAG 시스템의 성능을 수치로 증명
2. **리스크 관리**: 잘못된 정보 제공 시 발생하는 비용 최소화
3. **지속적 개선**: A/B 테스트를 통한 점진적 향상
4. **고객 신뢰**: 일관된 품질 보장

---

## 학습 목표

이 노트북에서는 다음을 배웁니다:

1. **테스트 데이터 구조** 설계 방법
2. **답변 평가 (Answer Evaluation)**: LLM-as-a-Judge 패러다임
3. **실전 평가** 파이프라인 구현
4. **종합 평가 리포트** 생성

> **참고**: 검색 평가 지표(MRR, nDCG, Keyword Coverage)는 **Part 1 (13-1.rag_evaluation_part1.ipynb)**에서 다룹니다.

---

## 1. 환경 설정

In [ ]:
# 필요한 라이브러리 설치
# !pip install pydantic litellm python-dotenv numpy

In [ ]:
import os
import json
import numpy as np
from typing import List, Dict, Optional, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

# API 키 확인
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API 키가 설정되었습니다.")
else:
    print("경고: OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")

---

## 2. 테스트 데이터 구조

### 왜 테스트 데이터가 필요한가?

RAG 시스템을 평가하려면 **정답이 있는 테스트 셋**이 필요합니다:

```
테스트 데이터 = 질문 + 예상 답변 + 관련 문서 정보
```

좋은 테스트 데이터의 특징:
- **다양한 유형**의 질문 포함
- **현실적인** 시나리오 반영
- **명확한 정답** 존재
- **충분한 양** (최소 50-100개 권장)

### TestQuestion 클래스 (Pydantic 모델)

In [ ]:
# 질문 카테고리 타입 정의
QuestionCategory = Literal[
    "direct_fact",    # 직접적인 사실 질문
    "temporal",       # 시간 관련 질문
    "spanning",       # 여러 문서에 걸친 질문
    "comparative",    # 비교 질문
    "numerical",      # 숫자/수치 질문
    "relationship",   # 관계 질문
    "holistic"        # 종합적 이해 질문
]


class TestQuestion(BaseModel):
    """
    RAG 평가를 위한 테스트 질문 모델
    
    각 필드 설명:
    - id: 질문 고유 식별자
    - question: 실제 질문 텍스트
    - expected_answer: 예상되는 정답 (참조 답변)
    - category: 질문 카테고리
    - source_docs: 정답을 찾을 수 있는 문서 ID들
    - keywords: 답변에 포함되어야 할 핵심 키워드들
    - difficulty: 난이도 (1: 쉬움, 2: 보통, 3: 어려움)
    """
    id: str = Field(..., description="질문 고유 식별자")
    question: str = Field(..., description="질문 텍스트")
    expected_answer: str = Field(..., description="예상 정답")
    category: QuestionCategory = Field(..., description="질문 카테고리")
    source_docs: List[str] = Field(default_factory=list, description="정답 소스 문서 ID들")
    keywords: List[str] = Field(default_factory=list, description="핵심 키워드들")
    difficulty: int = Field(default=1, ge=1, le=3, description="난이도 (1-3)")


# 예시 테스트 질문 생성
sample_question = TestQuestion(
    id="q001",
    question="Python에서 리스트를 정렬하는 방법은 무엇인가요?",
    expected_answer="Python에서 리스트를 정렬하려면 sort() 메서드 또는 sorted() 함수를 사용합니다. sort()는 원본 리스트를 직접 수정하고, sorted()는 새로운 정렬된 리스트를 반환합니다.",
    category="direct_fact",
    source_docs=["python_basics_001", "python_list_002"],
    keywords=["sort", "sorted", "리스트", "정렬"],
    difficulty=1
)

print("테스트 질문 예시:")
print(sample_question.model_dump_json(indent=2))

### 질문 카테고리 설명

다양한 카테고리의 질문으로 테스트해야 RAG 시스템의 전체적인 성능을 파악할 수 있습니다.

| 카테고리 | 설명 | 예시 | 권장 비율 |
|----------|------|------|----------|
| **direct_fact** | 직접적인 사실 질문 | "Python의 창시자는?" | 45-50% |
| **temporal** | 시간 관련 질문 | "Python 3.0은 언제 출시됐나?" | 10-15% |
| **spanning** | 여러 문서 연결 | "Django와 Flask의 차이점은?" | 10-15% |
| **comparative** | 비교 질문 | "어떤 방법이 더 빠른가?" | 5-10% |
| **numerical** | 수치 관련 | "최대 연결 수는?" | 5-10% |
| **relationship** | 관계 질문 | "A는 B와 어떤 관계?" | 5-10% |
| **holistic** | 종합 이해 | "전체 아키텍처를 설명해주세요" | 5-10% |

In [ ]:
# 샘플 테스트 데이터셋 생성
sample_test_data = [
    TestQuestion(
        id="q001",
        question="Python에서 리스트를 정렬하는 방법은?",
        expected_answer="sort() 메서드 또는 sorted() 함수를 사용합니다.",
        category="direct_fact",
        source_docs=["doc_001"],
        keywords=["sort", "sorted"],
        difficulty=1
    ),
    TestQuestion(
        id="q002",
        question="Python 3.0은 언제 출시되었나요?",
        expected_answer="Python 3.0은 2008년 12월 3일에 출시되었습니다.",
        category="temporal",
        source_docs=["doc_002"],
        keywords=["2008", "12월"],
        difficulty=1
    ),
    TestQuestion(
        id="q003",
        question="Django와 Flask의 주요 차이점은 무엇인가요?",
        expected_answer="Django는 풀스택 프레임워크로 ORM, 인증 등을 내장하고, Flask는 마이크로 프레임워크로 최소한의 기능만 제공하여 유연성이 높습니다.",
        category="spanning",
        source_docs=["doc_003", "doc_004"],
        keywords=["풀스택", "마이크로", "ORM", "유연성"],
        difficulty=2
    ),
    TestQuestion(
        id="q004",
        question="리스트 컴프리헨션과 for 루프 중 어떤 것이 더 빠른가요?",
        expected_answer="일반적으로 리스트 컴프리헨션이 for 루프보다 빠릅니다. 이는 내부적으로 최적화되어 있기 때문입니다.",
        category="comparative",
        source_docs=["doc_005"],
        keywords=["리스트 컴프리헨션", "빠르", "최적화"],
        difficulty=2
    ),
    TestQuestion(
        id="q005",
        question="Python의 기본 재귀 깊이 제한은 얼마인가요?",
        expected_answer="Python의 기본 재귀 깊이 제한은 1000입니다.",
        category="numerical",
        source_docs=["doc_006"],
        keywords=["1000", "재귀"],
        difficulty=1
    ),
]

print(f"테스트 데이터셋: {len(sample_test_data)}개 질문")
for q in sample_test_data:
    print(f"  [{q.category}] {q.question[:40]}...")

---

## 3. 검색 평가 (Retrieval Evaluation)

> **참고**: 검색 평가 지표(MRR, nDCG, Keyword Coverage)와 RetrievalEval 클래스는 
> **13-1.rag_evaluation_part1.ipynb**에서 상세히 다룹니다.
> 
> Part1에서 다루는 내용:
> - MRR (Mean Reciprocal Rank) - 이론용 및 실전용 함수
> - nDCG (Normalized DCG) - 이론용 및 실전용 함수
> - Keyword Coverage
> - RetrievalEval 종합 클래스
> - evaluate_retrieval 함수
>
> 아래 실습에서는 Part1에서 정의한 함수들을 사용합니다.

In [ ]:
# Part1에서 정의된 함수들 (인라인 정의)
# 이 셀은 Part1을 먼저 실행하지 않아도 이 노트북을 독립적으로 실행할 수 있도록 합니다.

def calculate_dcg(relevances: List[int], k: int = None) -> float:
    """DCG 계산"""
    if k is None:
        k = len(relevances)
    
    dcg = 0.0
    for i, rel in enumerate(relevances[:k], start=1):
        dcg += rel / np.log2(i + 1)
    return dcg


def calculate_mrr_practical(retrieved_docs: List[str], relevant_docs: List[str]) -> float:
    """MRR 계산 - 문서 ID 기반 실전 버전"""
    relevant_set = set(relevant_docs)
    for rank, doc_id in enumerate(retrieved_docs, start=1):
        if doc_id in relevant_set:
            return 1.0 / rank
    return 0.0


def calculate_ndcg_practical(retrieved_docs: List[str], relevant_docs: List[str], k: int = None) -> float:
    """nDCG 계산 - 문서 ID 기반 Binary Relevance 버전"""
    if k is None:
        k = len(retrieved_docs)
    
    relevant_set = set(relevant_docs)
    relevances = [1 if doc in relevant_set else 0 for doc in retrieved_docs[:k]]
    
    dcg = calculate_dcg(relevances, k)
    ideal_relevances = sorted(relevances, reverse=True)
    idcg = calculate_dcg(ideal_relevances, k)
    
    if idcg == 0:
        return 0.0
    return dcg / idcg


def calculate_keyword_coverage(retrieved_content: str, keywords: List[str]) -> float:
    """키워드 커버리지 계산"""
    if not keywords:
        return 1.0
    content_lower = retrieved_content.lower()
    found_count = sum(1 for kw in keywords if kw.lower() in content_lower)
    return found_count / len(keywords)


class RetrievalEval(BaseModel):
    """검색 평가 결과 모델"""
    mrr: float = Field(..., ge=0, le=1, description="Mean Reciprocal Rank")
    ndcg: float = Field(..., ge=0, le=1, description="Normalized DCG")
    keyword_coverage: float = Field(..., ge=0, le=1, description="키워드 커버리지")
    retrieved_count: int = Field(..., ge=0, description="검색된 문서 수")
    relevant_found: int = Field(..., ge=0, description="찾은 관련 문서 수")
    
    @property
    def overall_score(self) -> float:
        """종합 점수 (가중 평균)"""
        return 0.4 * self.mrr + 0.4 * self.ndcg + 0.2 * self.keyword_coverage


def evaluate_retrieval(
    retrieved_docs: List[str],
    relevant_docs: List[str],
    retrieved_content: str,
    keywords: List[str],
    k: int = 5
) -> RetrievalEval:
    """검색 평가 수행"""
    relevant_set = set(relevant_docs)
    relevant_found = sum(1 for doc in retrieved_docs if doc in relevant_set)
    
    return RetrievalEval(
        mrr=calculate_mrr_practical(retrieved_docs, relevant_docs),
        ndcg=calculate_ndcg_practical(retrieved_docs, relevant_docs, k),
        keyword_coverage=calculate_keyword_coverage(retrieved_content, keywords),
        retrieved_count=len(retrieved_docs),
        relevant_found=relevant_found
    )


class AnswerEval(BaseModel):
    """답변 평가 결과 모델"""
    accuracy: int = Field(..., ge=1, le=5, description="정확도 (1-5)")
    completeness: int = Field(..., ge=1, le=5, description="완전성 (1-5)")
    relevance: int = Field(..., ge=1, le=5, description="관련성 (1-5)")
    feedback: str = Field(..., description="상세 피드백")
    
    @property
    def average_score(self) -> float:
        """평균 점수"""
        return (self.accuracy + self.completeness + self.relevance) / 3
    
    @property
    def normalized_score(self) -> float:
        """정규화 점수 (0-1)"""
        return (self.average_score - 1) / 4


def evaluate_answer_with_llm(
    question: str,
    generated_answer: str,
    reference_answer: str,
    model: str = "gpt-4o-mini"
) -> AnswerEval:
    """LLM을 사용하여 답변 평가"""
    from litellm import completion
    
    prompt = f"""당신은 RAG 시스템의 답변 품질을 평가하는 전문 평가자입니다.

질문: {question}

생성된 답변: {generated_answer}

참조 답변: {reference_answer}

다음 기준으로 1-5점 (1: 매우 나쁨, 5: 매우 좋음) 평가해주세요:
1. 정확도 (accuracy): 사실적으로 얼마나 정확한가?
2. 완전성 (completeness): 질문의 모든 측면을 다루는가?
3. 관련성 (relevance): 질문에 직접적으로 답변하는가?

반드시 아래 JSON 형식으로만 응답하세요:
{{"accuracy": 점수, "completeness": 점수, "relevance": 점수, "feedback": "상세 피드백"}}"""
    
    response = completion(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"}
    )
    
    result = json.loads(response.choices[0].message.content)
    return AnswerEval(**result)


print("평가 함수들이 로드되었습니다.")

---

## 3. 답변 평가 (Answer Evaluation)

검색이 잘 되었더라도, **최종 답변의 품질**이 좋아야 합니다.

### 3.1 LLM-as-a-Judge 패러다임

**핵심 아이디어**: LLM을 평가자로 사용하여 답변 품질 측정

#### 장점
- **스케일러블**: 대량의 평가를 자동화
- **일관성**: 동일 기준으로 평가
- **세밀한 피드백**: 왜 좋은지/나쁜지 설명 가능
- **다양한 기준**: 정확도, 완전성, 관련성 등 여러 측면 평가

#### 단점
- **비용**: API 호출 비용 발생
- **편향 가능성**: LLM 자체의 편향 반영
- **일관성 한계**: 같은 입력에 다른 결과 가능

### 3.2 평가 기준 3가지

| 기준 | 설명 | 예시 질문 |
|------|------|----------|
| **Accuracy (정확도)** | 사실적으로 얼마나 정확한가? | "답변이 사실과 일치하나요?" |
| **Completeness (완전성)** | 질문의 모든 측면을 다루는가? | "모든 부분을 답변했나요?" |
| **Relevance (관련성)** | 질문에 직접적으로 답변하는가? | "질문에 맞는 답변인가요?" |

### 3.3 LLM Judge 구현

LiteLLM을 사용하여 다양한 LLM 제공자를 지원하는 Judge를 구현합니다.

---

## 4. 실습: 평가 실행

지금까지 배운 내용을 종합하여 실제 평가를 수행해봅니다.

In [ ]:
# 테스트 시나리오 설정
test_case = {
    "question": "Python에서 리스트를 정렬하는 방법은 무엇인가요?",
    "reference_answer": "Python에서 리스트를 정렬하려면 sort() 메서드 또는 sorted() 함수를 사용합니다. sort()는 원본 리스트를 직접 수정하고, sorted()는 새로운 정렬된 리스트를 반환합니다.",
    "keywords": ["sort", "sorted", "리스트", "정렬"],
    "relevant_docs": ["python_list_001", "python_basics_002"]
}

# 가상의 RAG 시스템 출력 (실제로는 RAG 파이프라인에서 생성)
rag_output = {
    "retrieved_docs": ["python_list_001", "python_intro_003", "python_basics_002", "django_001", "flask_002"],
    "retrieved_content": """Python에서 리스트 정렬은 sort() 메서드를 사용합니다. 
    이 메서드는 원본 리스트를 직접 수정합니다. sorted() 함수도 있습니다.""",
    "generated_answer": "Python에서 리스트를 정렬하려면 sort() 메서드를 사용합니다. 이 메서드는 원본 리스트를 직접 수정합니다."
}

print("테스트 케이스:")
print(f"  질문: {test_case['question']}")
print(f"  정답 문서: {test_case['relevant_docs']}")
print(f"\nRAG 출력:")
print(f"  검색된 문서: {rag_output['retrieved_docs']}")
print(f"  생성된 답변: {rag_output['generated_answer'][:50]}...")

In [ ]:
# 검색 평가 실행
retrieval_eval = evaluate_retrieval(
    retrieved_docs=rag_output["retrieved_docs"],
    relevant_docs=test_case["relevant_docs"],
    retrieved_content=rag_output["retrieved_content"],
    keywords=test_case["keywords"],
    k=5
)

print("=" * 60)
print("검색 평가 결과 (Retrieval Evaluation)")
print("=" * 60)
print(f"\nMRR: {retrieval_eval.mrr:.3f}")
print(f"  -> 첫 번째 정답(python_list_001)이 1번째에 있어 최고 점수")

print(f"\nnDCG@5: {retrieval_eval.ndcg:.3f}")
print(f"  -> 정답 2개(1,3번째)가 상위에 있어 좋은 점수")

print(f"\nKeyword Coverage: {retrieval_eval.keyword_coverage:.1%}")
print(f"  -> 4개 키워드 중 {int(retrieval_eval.keyword_coverage * 4)}개 발견")

print(f"\n관련 문서 발견: {retrieval_eval.relevant_found}/{len(test_case['relevant_docs'])}개")
print(f"\n종합 점수: {retrieval_eval.overall_score:.3f}")

In [ ]:
# 답변 평가 실행 (LLM API 호출)
# 주의: 이 셀은 API 키가 설정되어 있어야 실행됩니다.

try:
    answer_eval = evaluate_answer_with_llm(
        question=test_case["question"],
        generated_answer=rag_output["generated_answer"],
        reference_answer=test_case["reference_answer"],
        model="gpt-4o-mini"
    )
    
    print("=" * 60)
    print("답변 평가 결과 (Answer Evaluation by LLM Judge)")
    print("=" * 60)
    print(f"\n정확도 (Accuracy): {answer_eval.accuracy}/5")
    print(f"완전성 (Completeness): {answer_eval.completeness}/5")
    print(f"관련성 (Relevance): {answer_eval.relevance}/5")
    print(f"\n평균 점수: {answer_eval.average_score:.2f}/5")
    print(f"정규화 점수: {answer_eval.normalized_score:.2f}")
    print(f"\n피드백: {answer_eval.feedback}")
    
except Exception as e:
    print(f"API 호출 실패: {e}")
    print("\nAPI 키를 확인하거나, 아래 모의 결과를 참고하세요:")
    
    # 모의 결과
    mock_eval = AnswerEval(
        accuracy=4,
        completeness=3,
        relevance=5,
        feedback="답변이 정확하고 관련성이 높지만, sorted() 함수에 대한 설명이 누락되어 완전성이 다소 낮습니다."
    )
    print(f"\n[모의 결과]")
    print(f"정확도: {mock_eval.accuracy}/5")
    print(f"완전성: {mock_eval.completeness}/5")
    print(f"관련성: {mock_eval.relevance}/5")
    print(f"피드백: {mock_eval.feedback}")

---

## 5. 종합 평가 리포트 클래스

In [ ]:
class RAGEvaluationReport(BaseModel):
    """
    RAG 시스템 종합 평가 리포트
    """
    question_id: str
    question: str
    category: QuestionCategory
    retrieval: RetrievalEval
    answer: Optional[AnswerEval] = None
    
    def print_report(self):
        """평가 리포트 출력"""
        print("\n" + "=" * 70)
        print(f"RAG 평가 리포트: {self.question_id}")
        print("=" * 70)
        
        print(f"\n질문: {self.question}")
        print(f"카테고리: {self.category}")
        
        print("\n[검색 평가]")
        print(f"  MRR: {self.retrieval.mrr:.3f}")
        print(f"  nDCG: {self.retrieval.ndcg:.3f}")
        print(f"  Keyword Coverage: {self.retrieval.keyword_coverage:.1%}")
        print(f"  종합: {self.retrieval.overall_score:.3f}")
        
        if self.answer:
            print("\n[답변 평가]")
            print(f"  정확도: {self.answer.accuracy}/5")
            print(f"  완전성: {self.answer.completeness}/5")
            print(f"  관련성: {self.answer.relevance}/5")
            print(f"  평균: {self.answer.average_score:.2f}/5")
            print(f"\n  피드백: {self.answer.feedback}")
        
        print("\n" + "=" * 70)


# 종합 리포트 생성 예시
report = RAGEvaluationReport(
    question_id="q001",
    question=test_case["question"],
    category="direct_fact",
    retrieval=retrieval_eval,
    answer=AnswerEval(
        accuracy=4,
        completeness=3,
        relevance=5,
        feedback="답변이 정확하고 관련성이 높지만, sorted() 함수 설명이 누락됨"
    )
)

report.print_report()

---

## 6. 결론 및 Best Practices

### 핵심 정리

| 평가 단계 | 주요 지표 | 목적 |
|----------|----------|------|
| **검색 평가** | MRR, nDCG, Coverage | 올바른 문서를 찾았는가? |
| **답변 평가** | Accuracy, Completeness, Relevance | 올바른 답변을 생성했는가? |

### Best Practices

1. **평가 자동화**
   - CI/CD 파이프라인에 평가 포함
   - 배포 전 성능 회귀 테스트

2. **다양한 질문 카테고리**
   - direct_fact만으로는 부족
   - comparative, spanning 등 어려운 질문도 테스트

3. **정기적인 평가**
   - 문서 업데이트 시 재평가
   - 모델 변경 시 비교 평가

4. **평가 데이터 품질**
   - 전문가 검토된 테스트 셋
   - 실제 사용자 질문 반영

5. **개선 루프 구축**
   - 평가 -> 분석 -> 개선 -> 재평가
   - 실패 케이스 상세 분석

### 다음 단계

1. 더 큰 테스트 셋 구축 (100개 이상)
2. Ragas, DeepEval 등 전문 평가 프레임워크 활용
3. Human evaluation과 LLM evaluation 비교
4. 평가 대시보드 구축

---

## 참고 자료

- [Ragas: RAG 평가 프레임워크](https://github.com/explodinggradients/ragas)
- [DeepEval: LLM 평가 도구](https://github.com/confident-ai/deepeval)
- [LangChain Evaluation](https://python.langchain.com/docs/guides/evaluation)
- [BEIR Benchmark](https://github.com/beir-cellar/beir)

---

**이 노트북을 완료하셨습니다!**

이제 여러분은:
- 테스트 데이터를 설계할 수 있습니다
- 검색 품질을 MRR, nDCG로 평가할 수 있습니다
- LLM-as-a-Judge로 답변 품질을 평가할 수 있습니다
- 종합 평가 리포트를 생성할 수 있습니다